# Kitti数据集的使用方法

参考
https://www.youtube.com/playlist?list=PLDV2CyUo4q-L4YlXUWDytZPz9a8cAWXST


https://github.com/kwea123/ROS_notes


#下載kitti tracking資料並且讀取

资料信息
https://github.com/pratikac/kitti/blob/master/readme.tracking.txt

视频
https://www.youtube.com/watch?v=phyonnkuGbs&list=PLDV2CyUo4q-L4YlXUWDytZPz9a8cAWXST&index=7

In [2]:
import numpy as np
import pandas as pd

TRACKING_COLUMN_NAMES = ['frame', 'track_id', 'type', 'truncated', 'occluded', 'alpha', 'bbox_left', 'bbox_top',
                         'bbox_right', 'bbox_bottom', 'height', 'width', 'length', 'pos_x', 'pos_y', 'pos_z', 'rot_y']

def read_tracking(path):
    df = pd.read_csv(path, header=None, sep=' ')
    df.columns = TRACKING_COLUMN_NAMES
    #print(df.head())

    df = df[df['track_id']>=0] # remove DontCare objects
    df.loc[df.type.isin(['Bus', 'Truck', 'Van', 'Tram']), 'type'] = 'Car' # Set all vehicle type to Car
    df = df[df.type.isin(['Car', 'Pedestrian', 'Cyclist'])]
    return df

df = read_tracking("data/0000.txt")
read_tracking("data/0000.txt").head()
# bbox = np.array(df.loc[:, ['bbox_left',	'bbox_top',	'bbox_right',	'bbox_bottom']])
# print(bbox)


,frame,track_id,type,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,height,width,length,pos_x,pos_y,pos_z,rot_y
2,0,0,Car,0,0,-1.793451,296.744956,161.752147,455.226042,292.372804,2.000000,1.823255,4.433886,-4.552284,1.858523,13.410495,-2.115488
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245
7,1,0,Car,0,0,-1.796862,294.898777,156.024256,452.199718,284.621269,2.000000,1.823255,4.433886,-4.650955,1.766774,13.581085,-2.121565
8,1,1,Cyclist,0,0,-1.935205,745.017137,156.393157,938.839722,374.000000,1.739063,0.824591,1.785241,1.700640,1.640419,5.778596,-1.664456


In [5]:
# Get bounding box
df.loc[2, ['bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom']]

bbox_left      296.745
bbox_top       161.752
bbox_right     455.226
bbox_bottom    292.373
Name: 2, dtype: object

In [11]:
np.array(df[df.frame==0][['bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom']])

array([[ 296.744956,  161.752147,  455.226042,  292.372804],
       [ 737.619499,  161.531951,  931.112229,  374.      ],
       [1106.137292,  166.576807, 1204.470628,  323.876144]])

In [ ]:
import cv2
#from google.colab.patches import cv2_imshow

COLOR_DICT = {'Car':(255,255,0), 'Cyclist':(255,0,255), 'Pedestrian':(255,55,25)}

frame = 0
image = cv2.imread("data/%010d.png"%frame)
boxes = np.array(df[df.frame==0][['bbox_left','bbox_top','bbox_right','bbox_bottom']])
types = np.array(df[df.frame==0]['type'])

for typ, box in zip(types,boxes):
  top_left = int(box[0]),int(box[1])
  bottom_right = int(box[2]),int(box[3])
  cv2.rectangle(image, top_left, bottom_right,COLOR_DICT[typ],2)
#cv2_imshow(image)
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# lidar检测框

![](https://navoshta.com/images/posts/kitti-lidar/pcl_data.gif)

文章
https://navoshta.com/kitti-lidar/

代码
https://github.com/navoshta/KITTI-Dataset/blob/master/kitti-dataset.ipynb

In [12]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def draw_point_cloud(ax, points, axes=[0, 1, 2], point_size = 0.1, xlim3d=None, ylim3d=None, zlim3d=None):
    """
    Convenient method for drawing various point cloud projections as a part of frame statistics.
    """
    axes_limits = [
                   [-20,80],
                   [-20,20],
                   [-3,3]
    ]
    axes_str = ['X','Y','Z']
    ax.grid = False

    ax.scatter(*np.transpose(points[:, axes]), s=point_size, c=points[:, 3], cmap='gray')
    ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
    ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
    if len(axes) > 2:
        ax.set_xlim3d(*axes_limits[axes[0]])
        ax.set_ylim3d(*axes_limits[axes[1]])
        ax.set_zlim3d(*axes_limits[axes[2]])
        ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
    else:
        ax.set_xlim(*axes_limits[axes[0]])
        ax.set_ylim(*axes_limits[axes[1]])
    # User specified limits
    if xlim3d!=None:
        ax.set_xlim3d(xlim3d)
    if ylim3d!=None:
        ax.set_ylim3d(ylim3d)
    if zlim3d!=None:
        ax.set_zlim3d(zlim3d)


points = np.fromfile('%010d.bin'%frame,dtype=np.float32).reshape(-1,4)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(40,100)
draw_point_cloud(ax, points[::2]) #每2个点画一个

NameError: name 'frame' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
draw_point_cloud(ax, points[::2], axes=[0,1]) #鸟瞰

## 3d框
同样利用tracking文件，可以提取3d框信息。

The reference point for the 3D bounding box for each object is centered on the
bottom face of the box. 

The corners of bounding box are computed as follows with
respect to the reference point and in the object coordinate system:

```
x_corners = [l/2, l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2]^T
y_corners = [0,   0,    0,    0,   -h,   -h,   -h,   -h  ]^T
z_corners = [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2  ]^T
```
with l=length, h=height, and w=width.

讲解：https://www.youtube.com/watch?v=H8HGSIi77H0&list=PLDV2CyUo4q-L4YlXUWDytZPz9a8cAWXST&index=12


In [ ]:
def compute_3d_box_cam2(h, w, l, x, y, z, yaw):
    """
    Return : 3xn in cam2 coordinate
    """
    R = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
    x_corners = [l/2,l/2,-l/2,-l/2,l/2,l/2,-l/2,-l/2]
    y_corners = [0,0,0,0,-h,-h,-h,-h]
    z_corners = [w/2,-w/2,-w/2,w/2,w/2,-w/2,-w/2,w/2]
    corners_3d_cam2 = np.dot(R, np.vstack([x_corners,y_corners,z_corners]))
    corners_3d_cam2[0,:] += x
    corners_3d_cam2[1,:] += y
    corners_3d_cam2[2,:] += z
    return corners_3d_cam2

def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):
    """
    Draws a bounding 3D box in a pyplot axis.
    
    Parameters
    ----------
    pyplot_axis : Pyplot axis to draw in.
    vertices    : Array 8 box vertices containing x, y, z coordinates.
    axes        : Axes to use. Defaults to `[0, 1, 2]`, e.g. x, y and z axes.
    color       : Drawing color. Defaults to `black`.
    """
    vertices = vertices[axes, :]
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        pyplot_axis.plot(*vertices[:, connection], c=color, lw=0.5)

# 照相机坐标系
cornel_3d_cam2 = compute_3d_box_cam2(*df.loc[2,['height','width','length',	'pos_x','pos_y','pos_z','rot_y']])
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(40,150)
draw_box(ax, (cornel_3d_cam2)) #每2个点画一个

## 坐标系转换

3d偵測框的數學推導，並在照相機座標系裡將其畫出
另外介紹了calibration(校正)，使的各個座標系之間的點可以自由的進行轉換。我們於是可以把照相機座標系中的3d偵測框投影至velodyne座標系中，並與點雲同時畫出來


代码
https://github.com/charlesq34/frustum-pointnets/blob/master/kitti/kitti_util.py

In [ ]:
from  kitti_utils  import *

#读取kitti标定文件
calib = Calibration('utils',from_video=True)
# 从2号相机投影到激光传感器坐标系
cornel_3d_velo = calib.project_rect_to_velo(cornel_3d_cam2.T).T
print(cornel_3d_velo)
# 画出在激光坐标系下的方框
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(40,150)
draw_box(ax, (cornel_3d_velo)) #每2个点画一个

## 最终结果

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(40,150)
draw_point_cloud(ax, points[::2]) #每2个点画一个
draw_box(ax, (cornel_3d_velo)) #每2个点画一个

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
draw_point_cloud(ax, points[::2], axes=[0,1]) #鸟瞰
draw_box(ax, (cornel_3d_velo), axes=[0,1], color='r') #每2个点画一个

# 读取IMU GPS信息

oxts文件夹中,数据格式[参考如下](https://github.com/pratikac/kitti/blob/master/readme.raw.txt).

利用IMU/GPS資料計算移動距離以及旋轉角度

- 利用yaw得到旋转角度

- 移動距離有兩種算法
1.利用GPS計算大圓距離
https://zh.wikipedia.org/wiki/%E5%A4%A7%E5%9C%86%E8%B7%9D%E7%A6%BB

2.利用IMU提供的速度乘上經過的時間

在短時間內的話，以IMU資料計算出的結果會具有更高的精確性及平滑性。

In [ ]:
def read_imu(path):
  IMU_COLUMN_NAMES = ['lat', 'lon', 'alt', 'roll', 'pitch', 'yaw', 'vn', 've', 'vf', 'vl', 'vu', 'ax', 'ay', 'az', 'af',
                      'al', 'au', 'wx', 'wy', 'wz', 'wf', 'wl', 'wu', 'posacc', 'velacc', 'navstat', 'numsats', 'posmode',
                      'velmode', 'orimode']

  df = pd.read_csv(path, header=None, sep=' ')
  df.columns = IMU_COLUMN_NAMES
  return df
read_imu('/content/0000000000.txt')


In [ ]:
#利用gps计算两帧距离
def compute_great_circle_distance(lat1, lon1, lat2, lon2):
    """
    Compute the great circle distance from two gps data
    Input   : latitudes and longitudes in degree
    Output  : distance in meter
    """
    delta_sigma = float(np.sin(lat1*np.pi/180)*np.sin(lat2*np.pi/180)+ \
                        np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.cos(lon1*np.pi/180-lon2*np.pi/180))
    if np.abs(delta_sigma) > 1:
        return 0.0
    return 6371000.0 * np.arccos(delta_sigma)

# 计算所有
def compute_every_two_frame_distance():
  prev_imu_data = None
  gps_distances = []
  imu_distances = []
  for frame in range(130):
    imu_data = read_imu('%010d.txt'%frame)

    if prev_imu_data is not None:
      gps_distances += [compute_great_circle_distance(imu_data.lat, imu_data.lon, prev_imu_data.lat, prev_imu_data.lon)]
      imu_distances += [0.1*np.linalg.norm(imu_data[['vf','vl']])] # 0.1s移动距离
    prev_imu_data = imu_data
  return gps_distances, imu_distances

gps_distances,imu_distances = compute_every_two_frame_distance()
plt.figure()
plt.plot(gps_distances, label='gps')
plt.plot(imu_distances, label='imu')
plt.legend()
plt.show()

## 计算移动轨迹

https://www.youtube.com/watch?v=TEO3dFeIO-8&list=PLDV2CyUo4q-L4YlXUWDytZPz9a8cAWXST&index=16

In [ ]:
# 畫出自己所有過去的軌跡
def compute_locations():
  prev_imu_data = None
  locations = []
  for frame in range(130):
    imu_data = read_imu('%010d.txt'%frame)

    if prev_imu_data is not None:
      displacement = [0.1*np.linalg.norm(imu_data[['vf','vl']])] # 0.1s移动距离
      yaw_change = float(imu_data.yaw - prev_imu_data.yaw)
      for i in range(len(locations)):
        x0, y0 = locations[i]
        # 前一帧坐标变换到现在帧坐标
        x1 = x0*np.cos(yaw_change) + y0*np.sin(yaw_change) - displacement
        y1 = -x0*np.sin(yaw_change) + y0*np.cos(yaw_change)
        locations[i] = [x1,y1]

    locations += [[0,0]] #相对自己
    prev_imu_data = imu_data
  return locations

locations = compute_locations()
plt.figure()
plt.plot(np.array(locations)[:,0], np.array(locations)[:,1])

# 计算物体间距离

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def distance_point_to_segment(P,A,B):
  """
  calculates the min distance of point P to a segment AB.
  return min distance and point q
  """

  AP = P-A
  BP = P-B
  AB = B-A
  # 锐角，投影点在线段上
  if np.dot(AB,AP)>=0 and np.dot(-AB,BP)>=0:
    return np.abs(np.cross(AP,AB))/np.linalg.norm(AB), np.dot(AP,AB)/np.dot(AB,AB)*AB+A
  # 否则线段外
  d_PA = np.linalg.norm(AP)
  d_PB = np.linalg.norm(BP)
  if d_PA < d_PB:
    return d_PA, A 
  return d_PB, B

P = np.array([5,2],dtype=np.float64) #change Q
A = np.array([0,1],dtype=np.float64)
B = np.array([2,0],dtype=np.float64)

d, Q = distance_point_to_segment(P,A,B)

plt.axes().set_aspect('equal','datalim')
plt.scatter(P[0],P[1])
plt.plot(list(zip(A,B))[0], list(zip(A,B))[1],'r-o')

plt.plot(list(zip(P,Q))[0], list(zip(P,Q))[1],'y-')

In [ ]:
# 计算两个3d框的最短距离

def min_distance_cuboids(cub1,cub2):
  """
  compute min dist between two non-overlapping cuboids of shape (8,4)
  """

  minD = 1e5
  for i in range(4):
    for j in range(4):
      d, Q = distance_point_to_segment(cub1[i,:2], cub2[j,:2], cub2[j+1,:2])
      if d < minD:
        minD = d
        minP = cub1[i,:2]
        minQ = Q
  for i in range(4):
    for j in range(4):
      d, Q = distance_point_to_segment(cub1[i,:2], cub2[j,:2], cub2[j+1,:2])
      if d < minD:
        minD = d
        minP = cub2[i,:2]
        minQ = Q
  return minP, minQ, minD

corners_3d_velo = np.array([
  [15.12424719, 16.06928938, 12.27779014, 11.33274795, 15.10334837, 16.04839056,
  12.25689133, 11.31184914], 
  [ 6.49985556,  4.9406537,  2.64287284,  4.2020747, 6.47872485, 4.91952299,
   2.62174213, 4.18094399],
  [-1.70710774, -1.71370686, -1.7776097,  -1.77101058,  0.29267156,  0.28607244,
   0.2221696,   0.22876872]])

corners_3d_velo = corners_3d_velo.T
ego_car = np.array([[2.15, 0.9, -1.73], [2.15, -0.9, -1.73], [-1.95, -0.9, -1.73],[-1.95,0.9,-1.73], [2.15,0.9,-0.23], [2.15,-0.9,-0.23],[-1.95,-0.9,-0.23],[-1.95,0.9,-0.23]]
                   )

plt.axes().set_aspect('equal','datalim')
plt.plot(ego_car[:5,0],ego_car[:5,1],'-o')
plt.plot(corners_3d_velo[:5,0], corners_3d_velo[:5,1],'-o')
minP,minQ,minD = min_distance_cuboids(ego_car, corners_3d_velo)
plt.plot(list(zip(minP,minQ))[0], list(zip(minP,minQ))[1],'y-')